In [3]:
from owlready2 import *
import rdflib

You should consider upgrading via the 'c:\users\simon\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.
* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [4]:
#onto = get_ontology("file:///content/ITO.owl")
onto = get_ontology("https://github.com/OpenBioLink/ITO/raw/master/ITO.owl")
onto.load()
ITO = onto.get_namespace("https://ai-strategies.org/ontology/")
EDAM = onto.get_namespace("http://edamontology.org/")

In [5]:
graph = default_world.as_rdflib_graph()

In [6]:
import pandas as pd
import json
def sparql_result_to_df(sparql_result):
    # converts SPARQL SELECT results to Pandas dataframe
    
    cols = sparql_result.bindings[0].keys()

    out = []
    for row in sparql_result.bindings:
        item = []
        for c in cols:
            item.append(row.get(c, {}))
        out.append(item)
    return pd.DataFrame(out, columns=cols)

In [8]:
namespaces ={ 'owl': rdflib.OWL,
              'rdfs': rdflib.RDFS,
              'edam': 'http://edamontology.org/',
              'ito': 'https://ai-strategies.org/ontology/'}

In [32]:
result = graph.query("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX ito: <https://identifiers.org/ito:>
    SELECT DISTINCT ?parent_name ?child_name
    WHERE
    {
        ?child rdfs:subClassOf+ ito:Process .
        ?child rdfs:subClassOf ?parent .
        ?child rdfs:label ?child_name .
        ?parent rdfs:label ?parent_name .
        FILTER NOT EXISTS { ?child rdfs:subClassOf+ ito:Benchmarking } .
    }
""", initNs = namespaces)
df = sparql_result_to_df(result)
df = json.loads(df.to_json(orient='split'))["data"]

with open("sunburst.json", 'w') as outfile:
    json.dump(df, outfile)  